<a href="https://colab.research.google.com/github/annita049/ICT-6115-advanced-cryptography-/blob/main/novel_otp_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import secrets
import sys

class DeniableOTP:
    def __init__(self, encoding='utf-8'):
        self.encoding = encoding

    def _to_bytes(self, data):
        """Helper to ensure data is in bytes."""
        if isinstance(data, str):
            return data.encode(self.encoding)
        return data

    def generate_key(self, length):
        """
        Generates a truly random key of 'length' bytes.
        CRITICAL: Uses 'secrets' module for cryptographic strength.
        """
        return secrets.token_bytes(length)

    def xor_bytes(self, bytes_a, bytes_b):
        """Performs XOR operation on two byte sequences."""
        if len(bytes_a) != len(bytes_b):
            raise ValueError("Data and Key must be of equal length for OTP.")

        # Zip the two byte arrays and XOR them integer by integer
        return bytes([b1 ^ b2 for b1, b2 in zip(bytes_a, bytes_b)])

    def encrypt(self, plaintext):
        """Encrypts plaintext and returns (ciphertext, key)."""
        p_bytes = self._to_bytes(plaintext)

        # 1. Generate Key (Must be truly random and same length)
        key = self.generate_key(len(p_bytes))

        # 2. XOR to create Ciphertext
        ciphertext = self.xor_bytes(p_bytes, key)

        return ciphertext, key

    def decrypt(self, ciphertext, key):
        """Decrypts ciphertext using the provided key."""
        # XORing again reverses the process
        decrypted_bytes = self.xor_bytes(ciphertext, key)
        return decrypted_bytes.decode(self.encoding)

    def forge_key(self, ciphertext, fake_message):
        """
        NOVEL FEATURE: Generates a key that will decrypt the
        EXISTING ciphertext into a SPECIFIED fake message.
        """
        f_bytes = self._to_bytes(fake_message)

        if len(f_bytes) != len(ciphertext):
            raise ValueError(f"Fake message must be exactly {len(ciphertext)} bytes to match ciphertext.")

        # Math: K_fake = Ciphertext XOR M_fake
        fake_key = self.xor_bytes(ciphertext, f_bytes)
        return fake_key

# --- Demonstration ---

if __name__ == "__main__":
    otp = DeniableOTP()

    # 1. The Real Secret
    real_msg = "Attack at dawn"
    print(f"Original Message: {real_msg}")

    # 2. Encryption
    cipher, real_key = otp.encrypt(real_msg)
    print(f"Ciphertext (Hex): {cipher.hex()}")

    # 3. Prove it works
    decrypted = otp.decrypt(cipher, real_key)
    print(f"Decrypted with Real Key: {decrypted}")

    print("-" * 30)

    # 4. THE NOVEL SCENARIO: You are caught!
    # You need the ciphertext to look like a harmless grocery list.
    decoy_msg = "Buy milk eggs" # Note: Must be same length as original

    # Check lengths (Padding would be added in a full production system)
    diff = len(real_msg) - len(decoy_msg)
    if diff > 0:
        decoy_msg += " " * diff
    elif diff < 0:
        print("Error: Decoy is longer than original.")
        sys.exit()

    print(f"Attempting to forge key for decoy: '{decoy_msg}'")

    # 5. Generate the Fake Key
    fake_key = otp.forge_key(cipher, decoy_msg)

    # 6. Verify the 'Lie'
    # The adversary uses the fake key on the ORIGINAL ciphertext
    hackers_result = otp.decrypt(cipher, fake_key)

    print(f"Decrypted with Fake Key: {hackers_result}")

    # 7. Proof of Deniability
    print(f"\nIs Fake Key same as Real Key? {fake_key == real_key}")
    print("mathematically indistinguishable plausibility achieved.")

Original Message: Attack at dawn
Ciphertext (Hex): 96d6de2215ce68b298af3bcad4ea
Decrypted with Real Key: Attack at dawn
------------------------------
Attempting to forge key for decoy: 'Buy milk eggs '
Decrypted with Fake Key: Buy milk eggs 

Is Fake Key same as Real Key? False
mathematically indistinguishable plausibility achieved.
